In [1]:
# Параметры
BATCH_SIZE = 16384
model_path = '3.1.3_DCN_MLP.pth'
test_csv = 'test_pairs.csv'  # Путь к тестовым данным
models_folder = 'C:/Users/Николай/PycharmProjects/VKRecSys/B.Processing/Модели/'
custom_data_folder = 'C:/Users/Николай/PycharmProjects/VKRecSys/custom_data/'
data_folder = 'C:/Users/Николай/PycharmProjects/VKRecSys/data/'
test_output_path = '3.1.3_predictions.csv' 
results_folder = 'C:/Users/Николай/PycharmProjects/VKRecSys/C.Results/'
orig_data_folder = 'C:/Users/Николай/PycharmProjects/VKRecSys/data/'

NUM_CROSS_LAYERS = 1

In [2]:
import numpy as np
import torch
import torch.nn.functional as F
import pandas as pd
from tqdm import tqdm
import torch.nn as nn

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

test = pd.read_csv(f'{orig_data_folder}{test_csv}')

In [4]:
items_meta = pd.read_parquet(f'{orig_data_folder}items_meta.parquet', engine='pyarrow')
items_meta['item_id'] = items_meta['item_id'].astype('category')
items_meta['source_id'] = items_meta['source_id'].astype('category')
items_meta.set_index('item_id', inplace=True)

users_meta = pd.read_parquet(f'{orig_data_folder}users_meta.parquet', engine='pyarrow')
users_meta['user_id'] = users_meta['user_id'].astype('category')
users_meta['gender'] = users_meta['gender'].astype('category')
users_meta['age'] = users_meta['age'].replace({1: 0, 2: 1})
users_meta.set_index('user_id', inplace=True)

In [5]:
def normalize_column(column):
    return (column - column.min()) / (column.max() - column.min())


users_meta['age'] = normalize_column(users_meta['age'])
items_meta['duration'] = normalize_column(items_meta['duration'])

In [6]:
item_embeddings_dict = items_meta['embeddings'].to_dict()


class EmbeddingLayer(nn.Module):
    def __init__(self, num_embeddings, embedding_dim):
        super(EmbeddingLayer, self).__init__()
        self.embedding = nn.Embedding(num_embeddings, embedding_dim)

    def forward(self, x):
        return self.embedding(x)

In [7]:
state = torch.load(f'{models_folder}{model_path}', map_location=device)

C:\Users\Николай\AppData\Local\Temp\ipykernel_18000\605623018.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state = torch.load(f'{models_folder}{model_path}', map_loca

In [8]:
num_users = users_meta.index.nunique()
num_items = items_meta.index.nunique()
num_sources = items_meta['source_id'].nunique()

In [9]:
user_embedding = EmbeddingLayer(num_users, 256).to(device)
user_embedding.load_state_dict(state['user_embedding_state_dict'])

item_embedding = EmbeddingLayer(num_items, 256).to(device)
item_embedding.load_state_dict(state['item_embedding_state_dict'])

source_embedding = EmbeddingLayer(num_sources, 256).to(device)
source_embedding.load_state_dict(state['source_embedding_state_dict'])

<All keys matched successfully>

In [10]:
input_dim = (1 + # gender
             1 + # age
             1 + # duration
             user_embedding.embedding.embedding_dim + 
             item_embedding.embedding.embedding_dim + 
             source_embedding.embedding.embedding_dim +
             32 # embeddings
             )

In [11]:
class DCN(nn.Module):
    def __init__(self, input_dim, num_cross_layers):
        super(DCN, self).__init__()
        self.input_dim = input_dim
        self.num_cross_layers = num_cross_layers
        
        # Параметры для слоев пересечения
        self.cross_weights = nn.ParameterList(
            [nn.Parameter(torch.randn(input_dim, 1)) for _ in range(num_cross_layers)]
        )
        self.cross_biases = nn.ParameterList(
            [nn.Parameter(torch.randn(input_dim)) for _ in range(num_cross_layers)]
        )
        
    def forward(self, x):
        # Инициализируем x0
        x0 = x
        for i in range(self.num_cross_layers):
            x = x0 * (x @ self.cross_weights[i]) + self.cross_biases[i] + x
        return x

class DCNWithMLP(nn.Module):
    def __init__(self, input_dim, num_cross_layers=3, hidden_dim=2048, output_dim=3):
        super(DCNWithMLP, self).__init__()
        
        # Нормализация входных данных
        self.batch_norm = nn.BatchNorm1d(input_dim)
        
        # DCN модуль
        self.dcn = DCN(input_dim, num_cross_layers)
        
        # MLP модуль
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, 2048)
        self.fc3 = nn.Linear(2048, 1024)
        self.fc4 = nn.Linear(1024, 512)
        self.fc5 = nn.Linear(512, 512)
        self.fc6 = nn.Linear(512, 256)
        self.fc7 = nn.Linear(256, 256)
        self.fc8 = nn.Linear(256, 128)
        self.fc9 = nn.Linear(128, output_dim)
        self.softplus = nn.Softplus()
    
    def forward(self, x):
        # Применяем нормализацию входных данных
        # x = self.batch_norm(x)
        
        # Пропускаем через DCN
        # x = self.dcn(x)
        
        # Пропускаем через MLP
        x = self.softplus(self.fc1(x))
        x = self.softplus(self.fc2(x))
        x = self.softplus(self.fc3(x))
        x = self.softplus(self.fc4(x))
        x = self.softplus(self.fc5(x))
        x = self.softplus(self.fc6(x))
        x = self.softplus(self.fc7(x))
        x = self.softplus(self.fc8(x))
        x = self.fc9(x)
        return x

# Определяем параметры
num_cross_layers = NUM_CROSS_LAYERS  # Количество слоев DCN

In [12]:
model = DCNWithMLP(input_dim, NUM_CROSS_LAYERS).to(device)
model.load_state_dict(state['model_state_dict'])
model.eval()

DCNWithMLP(
  (batch_norm): BatchNorm1d(803, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (dcn): DCN(
    (cross_weights): ParameterList(  (0): Parameter containing: [torch.float32 of size 803x1 (cuda:0)])
    (cross_biases): ParameterList(  (0): Parameter containing: [torch.float32 of size 803 (cuda:0)])
  )
  (fc1): Linear(in_features=803, out_features=2048, bias=True)
  (fc2): Linear(in_features=2048, out_features=2048, bias=True)
  (fc3): Linear(in_features=2048, out_features=1024, bias=True)
  (fc4): Linear(in_features=1024, out_features=512, bias=True)
  (fc5): Linear(in_features=512, out_features=512, bias=True)
  (fc6): Linear(in_features=512, out_features=256, bias=True)
  (fc7): Linear(in_features=256, out_features=256, bias=True)
  (fc8): Linear(in_features=256, out_features=128, bias=True)
  (fc9): Linear(in_features=128, out_features=3, bias=True)
  (softplus): Softplus(beta=1.0, threshold=20.0)
)

In [13]:
num_samples = len(test)
num_batches = (num_samples + BATCH_SIZE - 1) // BATCH_SIZE

In [14]:
predictions = []

In [15]:
with torch.no_grad():
    with tqdm(range(num_batches), desc="Predicting", unit="batch") as t:
        for batch_idx in t:
            start_idx = batch_idx * BATCH_SIZE
            end_idx = min(start_idx + BATCH_SIZE, num_samples)

            batch = test.iloc[start_idx:end_idx]

            batch_user_ids = torch.tensor(batch['user_id'].values, dtype=torch.long, device=device)
            batch_item_ids = torch.tensor(batch['item_id'].values, dtype=torch.long, device=device)
            batch_source_ids = torch.tensor(items_meta.loc[batch['item_id'].values, 'source_id'].cat.codes.values, dtype=torch.long, device=device)
            batch_age_ids = torch.tensor(users_meta.loc[batch['user_id'].values, 'age'].values, dtype=torch.float32, device=device).unsqueeze(1)
            batch_duration_ids = torch.tensor(items_meta.loc[batch['item_id'].values, 'duration'].values, dtype=torch.float32, device=device).unsqueeze(1)
            batch_gender_ids = torch.tensor(users_meta.loc[batch['user_id'].values, 'gender'].cat.codes.values, dtype=torch.float32, device=device).unsqueeze(1)

            user_emb = user_embedding(batch_user_ids)
            item_emb = item_embedding(batch_item_ids)
            source_emb = source_embedding(batch_source_ids)

            item_indices = batch_item_ids.cpu().numpy()
            embeddings = torch.tensor(
                np.stack(items_meta.loc[item_indices, 'embeddings'].values), 
                device=device, 
                dtype=torch.float32
            )

            inputs = torch.cat((
                user_emb,
                item_emb,
                source_emb,
                batch_age_ids,
                batch_duration_ids,
                batch_gender_ids,
                embeddings
            ), dim=1).float()

            # Model inference
            outputs = model(inputs)
            probabilities = F.softmax(outputs, dim=1)

            # Weighted predictions
            class_weights = torch.tensor([0, 1, 2], device=probabilities.device, dtype=probabilities.dtype)
            weighted_predictions = torch.sum(probabilities * class_weights, dim=1).cpu().numpy()

            predictions.extend(weighted_predictions)

Predicting: 100%|██████████| 102/102 [00:23<00:00,  4.30batch/s]


In [16]:
test['predict'] = predictions

In [17]:
test.to_csv(f'{results_folder}{test_output_path}')
print(f"Predictions saved to f'{results_folder}{test_output_path}")

Predictions saved to f'C:/Users/Николай/PycharmProjects/VKRecSys/C.Results/3.1.3_predictions.csv
